In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls 'drive/My Drive/megathon_pwc'

drainage_data_high.csv	      drainage_data_low_large.csv
drainage_data_high_large.csv  three_layer_high_large.h5
drainage_data_low.csv	      two_layer_high_large.h5


In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import TimeDistributed
from keras.layers import Reshape
from keras.layers import Flatten
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
df = pd.read_csv('/content/drive/My Drive/megathon_pwc/drainage_data_high_large.csv')

In [0]:
df.columns=['Date','Gas','Flow','Level','Temperature','Label']
x = df[['Gas','Flow','Level','Temperature']]
y = df[['Label']]

In [0]:
y_arr = y.to_numpy()
y_arr = np.reshape(y_arr,(y_arr.shape[0],))

In [0]:
y_arr = y_arr - 1
y_data = np.zeros((y_arr.shape[0], 4))
y_data[np.arange(y_arr.shape[0]), y_arr] = 1

In [0]:
x_np = x.to_numpy()
x_np = x_np - x_np.mean()
x_np = x_np/x_np.max()

In [0]:
x_new = np.reshape(x_np,(int(x_np.shape[0]/5),5,x_np.shape[1]))    
y_new = np.reshape(y_data,(int(y_data.shape[0]/5),5,y_data.shape[1]))

In [0]:
X_train, X_validation, y_train, y_validation = train_test_split(
    x_new,
    y_new,
    test_size=0.2,
    random_state=4)

In [11]:
X_validation.shape

(200, 5, 4)

In [12]:
model = Sequential()

In [13]:
model.add(LSTM(5,batch_input_shape=(None,5,4),return_sequences=True))
model.add(LSTM(5,batch_input_shape=(None,5,4),return_sequences=True,dropout=0.5))
model.add(LSTM(5,batch_input_shape=(None,5,4),return_sequences=True))
model.add(TimeDistributed(Dense(4,activation='softmax')))




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 5, 5)              200       
_________________________________________________________________
lstm_2 (LSTM)                (None, 5, 5)              220       
_________________________________________________________________
lstm_3 (LSTM)                (None, 5, 5)              220       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 4)              24        
Total params: 664
Trainable params: 664
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [0]:
# X_train = np.expand_dims(X_train,axis=3)
history = model.fit(X_train,y_train,epochs=1000,validation_data=(X_validation,y_validation))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 800 samples, validate on 200 samples
Epoch 1/1000
800/800 [==============================] - 2s 3ms/step - loss: 1.3773 - acc: 0.3633 - val_loss: 1.3652 - val_acc: 0.3850
Epoch 2/1000
800/800 [==============================] - 0s 411us/step - loss: 1.3559 - acc: 0.3790 - val_loss: 1.3399 - val_acc: 0.3850
Epoch 3/1000
800/800 [==============================] - 0s 406us/step - loss: 1.3321 - acc: 0.3790 - val_loss: 1.3105 - val_acc: 0.3850
Epoch 4/1000
800/800 [==============================] - 0s 422us/step - loss: 1.3088 - acc: 0.3790 - val_loss: 1.2900 - val_acc: 0.3850
Epoch 5/1000
800/800 [==============================] - 0s 416us/step - loss: 1.2982 - acc: 0.3790 - val_loss: 1.2835 - val_acc: 0.3850
Epoch 6/1000
800/800 [==============================] - 0s 428us/step - loss: 1.2933 - acc: 0.3790 - val_loss: 1.2804 - val_acc: 0.3850
Epoch 7/1000
800/800 [========================

In [0]:
results = model.predict_classes(X_validation)

In [0]:
results

In [0]:
model.save('/content/drive/My Drive/megathon_pwc/three_layer_high_large_dropout.h5')